Try to pull down all the Five Guys and In N Outs in CA from Yelp API
Look at ratings across the state for these chains...how much do they vary,
which one has more ratings and which overall has higher ratings?

In [1]:
import requests
import pandas as pd

In [2]:
import json

#Our previous function for loading our api key file
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [3]:
keys = get_keys("/Users/morgandj/.secret/creds.json")

api_key = keys['key']


In [31]:
def prepare_data(data_list):
    """
    This function takes in a list of dictionaries and prepares it
    for analysis
    """
    
    # Make a new list to hold results
    results = []
    
    for business_data in data_list:
    
        # Make a new dictionary to hold prepared data for this business
        prepared_data = {}
        
        # Extract name, review_count, rating, and price key-value pairs
        # from business_data and add to prepared_data
        # If a key is not present in business_data, add it to prepared_data
        # with an associated value of None
        bus_keys=business_data.keys()
        keys=['name','review_count','rating','price','id']
        for k in keys:
            if k in bus_keys:
                prepared_data[k]=business_data[k]
            else:
                prepared_data[k]=None
    
        # Parse and add latitude and longitude columns
        if 'coordinates' in bus_keys:
            coords=business_data['coordinates']
            prepared_data['latitude']=coords['latitude']
            prepared_data['longitude']=coords['longitude']
        else:
            prepared_data['latitude']=None
            prepared_data['longitude']=None
            
        #Add state for my own analysis
        if 'location' in bus_keys:
            prepared_data['city']=business_data['location']['city']
            prepared_data['state']=business_data['location']['state']
            prepared_data['zip_code']=business_data['location']['zip_code']
        else:
            prepared_data['state']=None
            prepared_data['city']=None
            prepared_data['zip_code']=None
        
        # Add to list if all values are present
        if all(prepared_data.values()):
            results.append(prepared_data)
    
    return results

In [23]:
url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': 'Five+Guys',
                'location': '92104'
            }

In [24]:
response=requests.get(url,headers=headers,params=url_params)
response.status_code

200

In [27]:
resp_json=response.json()
resp_json.keys()

dict_keys(['businesses', 'total', 'region'])

In [25]:
resp_json['total']

4500

In [32]:
prepared=prepare_data(resp_json['businesses'])
df=pd.DataFrame(prepared)
df

,name,review_count,rating,price,id,latitude,longitude,city,state,zip_code
0,Five Guys,318,3.5,$$,Xp4JLy_IE636zQqplp2w5A,32.748703,-117.158792,San Diego,CA,92103
1,Five Guys,195,3.5,$$,4fTOcIrimpyzqytLje18Tg,32.820620,-117.178996,San Diego,CA,92117
2,Five Guys,436,3.0,$$,oOj2yWSu1F-hCmy0C79npQ,32.867710,-117.214910,San Diego,CA,92122
3,Five Guys,54,3.5,$,JeJChOgVwliEqazlmCMk2Q,32.689997,-117.128225,San Diego,CA,92136
4,Five Guys,470,3.5,$$,N7hWCHXyPCUUXF-CAoMSeA,32.736750,-117.216375,San Diego,CA,92106
5,Five Guys,73,3.5,$$,QfH5naEmDdCPzlXXpZG6mQ,32.700355,-117.180176,San Diego,CA,92135
6,Five Guys,50,2.5,$$,cNg5tajDEy7jNxtviQzwoA,32.630410,-117.087458,Chula Vista,CA,91910
7,Baba's Chicken,167,4.0,$$,jotUXdeCamas8hX7UlbWPg,32.748379,-117.161420,San Diego,CA,92103
8,Canada Steak Burger,858,4.5,$,fncYUHs86Ls615a_UGbS9A,32.749940,-117.115360,San Diego,CA,92104
9,Shake Shack,171,3.5,$$,uWY3YyklKbnXkKcFXkJxaA,32.725540,-117.169360,San Diego,CA,92101


From this...would want to filter for just the Five Guys. Could consider removing any zip codes from the search that were returned, but then you could still miss other restaurants in the same zip code...

Would then add/extend this to existing list...would have to decide whether to do this all in json format or pandas..